## PCA from Scratch: Introduction

In this notebook, we implement Principal Component Analysis (PCA) step by step, without using any built-in PCA functions.

We use a small dataset of a student's grades across five subjects to:

- Understand how PCA works mathematically,
- Perform each step manually: standardization, covariance, eigen decomposition, projection,
- Compare our results with scikit-learn’s PCA to validate correctness.

The goal is to build a solid understanding of PCA by reproducing it from scratch.

### Generate Data

In [1]:
import pandas as pd
import numpy as np

data = {
    "Math": [80, 85, 78, 92, 70],
    "Physics": [78, 82, 76, 88, 72],
    "Chemistry": [65, 70, 60, 75, 58],
    "Biology": [60, 65, 58, 68, 55],
    "History": [70, 75, 72, 78, 65]
}

df = pd.DataFrame(data)
display(df)

,Math,Physics,Chemistry,Biology,History
0,80,78,65,60,70
1,85,82,70,65,75
2,78,76,60,58,72
3,92,88,75,68,78
4,70,72,58,55,65


### Step 1: Standardize the Data

PCA works best when the data has a mean of 0 and a standard deviation of 1.  
This makes sure each feature has equal importance, no matter its original scale.

The formula for standardization is:

$$
z = \frac{x - \mu}{\sigma}
$$

Where:

- `x` is the original value  
- `μ` is the mean of the column  
- `σ` is the standard deviation of the column

In [2]:
standardized_rows = []

for idx, row in df.iterrows():
    standardized_row = {}
    for col in df.columns:
        col_values = df[col]
        mean = col_values.sum() / len(col_values)
        variance = ((col_values - mean) ** 2).sum() / (len(col_values) - 1)
        std = variance ** 0.5
        z = (row[col] - mean) / std
        standardized_row[col] = z
    standardized_rows.append(standardized_row)

df_standardized = pd.DataFrame(standardized_rows)
display(df_standardized)

,Math,Physics,Chemistry,Biology,History
0,-0.122169,-0.196748,-0.085453,-0.228003,-0.404061
1,0.488678,0.459078,0.626656,0.722011,0.606092
2,-0.366508,-0.524661,-0.797562,-0.608009,0.000000
3,1.343864,1.442817,1.338765,1.292019,1.212183
4,-1.343864,-1.180487,-1.082406,-1.178018,-1.414214


### Step 2: Compute the Covariance Matrix

Now that the data is standardized, we can compute the covariance matrix.

It shows how two features change together.  
A high positive value means they increase together.  
A negative value means when one increases, the other decreases.

The formula is:

$$
\text{Cov}(X) = \frac{1}{n - 1} \cdot X^T X
$$

Where:
- `X` is the standardized data (mean = 0, std = 1),
- `n` is the number of samples (students),
- `Xᵀ` is the transpose of `X`.

In [3]:
n_samples = len(df_standardized)

cov_matrix = []

for col1 in df_standardized.columns:
    row = []
    for col2 in df_standardized.columns:
        x = df_standardized[col1]
        y = df_standardized[col2]
        product_sum = sum((x[i] * y[i]) for i in range(n_samples))
        covariance = product_sum / (n_samples - 1)
        row.append(covariance)
    cov_matrix.append(row)

cov_df = pd.DataFrame(cov_matrix, columns=df_standardized.columns, index=df_standardized.columns)
display(cov_df)

,Math,Physics,Chemistry,Biology,History
Math,1.000000,0.991508,0.965678,0.980730,0.968767
Physics,0.991508,1.000000,0.983076,0.987525,0.944040
Chemistry,0.965678,0.983076,1.000000,0.990416,0.891980
Biology,0.980730,0.987525,0.990416,1.000000,0.940466
History,0.968767,0.944040,0.891980,0.940466,1.000000


### Step 3: Eigen Decomposition

Now we will find the **eigenvalues** and **eigenvectors** of the covariance matrix.

- **Eigenvectors** show the directions of the new feature space (principal components).
- **Eigenvalues** show how much variance is explained in each direction.

We will sort the eigenvalues from largest to smallest. The directions (eigenvectors) with the highest eigenvalues represent the most important components in our data.

This is the main idea of PCA:  
We find new directions (axes) that capture the most variation in the data.

In [4]:
eigenvalues, eigenvectors = np.linalg.eig(cov_df.values)

eigenvalues = eigenvalues.real
eigenvectors = eigenvectors.real

eigen_df = pd.DataFrame({
    "Eigenvalue": eigenvalues,
    "Explained Variance (%)": 100 * eigenvalues / sum(eigenvalues)
})

display(eigen_df)

,Eigenvalue,Explained Variance (%)
0,4.858497e+00,9.716994e+01
1,1.199064e-01,2.398127e+00
2,1.737679e-02,3.475357e-01
3,4.219662e-03,8.439325e-02
4,-1.224759e-16,-2.449519e-15


### Step 4: Sort and Select Principal Components

Now that we have the eigenvalues and eigenvectors:

- We sort the eigenvalues from largest to smallest.
- The eigenvectors are reordered in the same way.
- The first principal component is the one with the largest eigenvalue.
- The number of components to keep depends on how much variance we want to keep.

This step helps us decide how many components we need to represent most of the data.

This is important:

- "Principal" means main or most important.
- A "component" is a direction (a vector) in the data space.

So, a **principal component** is the direction that captures the most variation in the data —  
in other words, the strongest pattern in the dataset.

In [5]:
sorted_indices = np.argsort(eigenvalues)[::-1]
sorted_eigenvalues = eigenvalues[sorted_indices]
sorted_eigenvectors = eigenvectors[:, sorted_indices]

explained_variance_ratio = sorted_eigenvalues / np.sum(sorted_eigenvalues)

df_explained = pd.DataFrame({
    "Eigenvalue": sorted_eigenvalues,
    "Explained Variance (%)": explained_variance_ratio * 100
})
display(df_explained)

,Eigenvalue,Explained Variance (%)
0,4.858497e+00,9.716994e+01
1,1.199064e-01,2.398127e+00
2,1.737679e-02,3.475357e-01
3,4.219662e-03,8.439325e-02
4,-1.224759e-16,-2.449519e-15


Now we move the standardized data into a new space using the principal components.

We do this by multiplying the data with the selected eigenvectors.

The result is a new dataset:
- The first column shows each data point’s value on the first principal component,
- The second column shows its value on the second component, and so on.

This reduces the number of features while keeping most of the important information.

In [6]:
X_std = df_standardized.values
principal_components = np.dot(X_std, sorted_eigenvectors)
df_pca = pd.DataFrame(principal_components, columns=[f"PC{i+1}" for i in range(principal_components.shape[1])])
display(df_pca)

,PC1,PC2,PC3,PC4,PC5
0,0.461337,-0.209666,0.100357,0.096568,1.651595e-15
1,-1.297141,-0.023220,-0.217338,0.023499,7.288014e-16
2,1.031525,0.591873,0.031864,-0.002055,4.046859e-16
3,-2.966256,-0.078556,0.104048,-0.054910,8.469801e-16
4,2.770535,-0.280431,-0.018932,-0.063102,9.825024e-16


## Comparison: Manual PCA vs. Scikit-Learn PCA

Now that we have completed PCA from scratch, we will compare our results with the output of `sklearn.decomposition.PCA`.

We will:
- Apply PCA using scikit-learn,
- Check the explained variance,
- Compare the transformed values (principal components).

If everything is correct, both methods should give the same results (up to numerical precision).

In [7]:
from sklearn.decomposition import PCA


pca = PCA()
X_sklearn = pca.fit_transform(df_standardized)
df_sklearn_pca = pd.DataFrame(X_sklearn, columns=[f"PC{i+1}" for i in range(X_sklearn.shape[1])])

aligned_manual = df_pca.copy()

for i in range(df_pca.shape[1]):
    if np.dot(df_pca.iloc[:, i], df_sklearn_pca.iloc[:, i]) < 0:
        aligned_manual.iloc[:, i] *= -1

manual_rounded = aligned_manual.round(5)
sklearn_rounded = df_sklearn_pca.round(5)
difference = (manual_rounded - sklearn_rounded).abs().add_suffix(" (abs diff)")

comparison = pd.concat([
    manual_rounded.add_suffix(" (manual aligned)"),
    sklearn_rounded.add_suffix(" (sklearn)"),
    difference
], axis=1)

display(comparison)

,PC1 (manual aligned),PC2 (manual aligned),PC3 (manual aligned),PC4 (manual aligned),PC5 (manual aligned),PC1 (sklearn),PC2 (sklearn),PC3 (sklearn),PC4 (sklearn),PC5 (sklearn),PC1 (abs diff),PC2 (abs diff),PC3 (abs diff),PC4 (abs diff),PC5 (abs diff)
0,-0.46134,-0.20967,-0.10036,-0.09657,0.0,-0.46134,-0.20967,-0.10036,-0.09657,0.0,0.0,0.0,0.0,0.0,0.0
1,1.29714,-0.02322,0.21734,-0.02350,0.0,1.29714,-0.02322,0.21734,-0.02350,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.03153,0.59187,-0.03186,0.00206,0.0,-1.03153,0.59187,-0.03186,0.00206,0.0,0.0,0.0,0.0,0.0,0.0
3,2.96626,-0.07856,-0.10405,0.05491,0.0,2.96626,-0.07856,-0.10405,0.05491,0.0,0.0,0.0,0.0,0.0,0.0
4,-2.77054,-0.28043,0.01893,0.06310,0.0,-2.77054,-0.28043,0.01893,0.06310,0.0,0.0,0.0,0.0,0.0,0.0


## Summary

We manually implemented all steps of Principal Component Analysis (PCA), including:

- Standardization
- Covariance matrix calculation
- Eigen decomposition
- Sorting components
- Projecting the data

Finally, we compared our manual results with `sklearn.PCA`. After aligning the component directions, the outputs matched exactly.

This confirms that our PCA implementation is correct.